In [1]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess
import numpy as np
import pandas as pd

In [2]:
# After uploading one part, eventually make a fake copy to work with two (identical) chunks
!cp part-00000.tsv part-00012.tsv

In [2]:
cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:41655 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 16.48 GB


In [3]:
features = [
    'text_tokens',
    'hashtags',
    'tweet_id',
    'media',
    'links',
    'domains',
    'tweet_type',
    'language',
    'timestamp',
    'a_user_id',
    'a_follower_count',
    'a_following_count',
    'a_is_verified',
    'a_account_creation',
    'b_user_id',
    'b_follower_count',
    'b_following_count',
    'b_is_verified',
    'b_account_creation',
    'b_follows_a',
    'reply',
    'retweet', 
    'retweet_comment',
    'like'
]

In [4]:
%%time
df = dask_cudf.read_csv('part-*', sep='\x01', names=features)
df.head()

CPU times: user 1.82 s, sys: 792 ms, total: 2.61 s
Wall time: 5.66 s


,text_tokens,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,...,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like
0,101\t1910\t15752\t39862\t22946\t1881\t1973\t15...,<NA>,083AA40FC7D55123858207B610D3C06C,<NA>,<NA>,<NA>,TopLevel,E7F038DE3EAD397AEC9193686C911677,1612436125,AFC3FC2E4F720A36A9B7068E372AACA8,...,51833CA723B31C534AD1FDA3896EAEEC,541,310,False,1351543406,True,<NA>,<NA>,<NA>,1.612443633e+09
1,101\t10657\t61047\t18426\t169\t10285\t54580\t1...,<NA>,C5E154739483D8733D68EC6FE8783C22,<NA>,<NA>,<NA>,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,1613276405,E27A4DB45665D5614B08EEE80A760EBA,...,258CADFCC98D233420B42FE6AD3EBD05,242,168,False,1348003745,True,<NA>,<NA>,<NA>,1.613278429e+09
2,101\t758\t13027\t11076\t791\t72929\t11145\t125...,<NA>,759D744337DC3404CFC4E4DF43BA961F,<NA>,<NA>,<NA>,TopLevel,00304D7356D6C64481190D708D8F739C,1614077871,32DF697D49138DC2161A5413068734CD,...,AE59B086B5E3954CD3034197D135C207,320,59,False,1553742110,False,<NA>,<NA>,<NA>,<NA>
3,101\t56898\t137\t13486\t17518\t10240\t10575\t2...,<NA>,C13243A3DADC6C2BCD4322100F3CC906,Video,<NA>,<NA>,Retweet,9FCF19233EAD65EA6E32C2E6DC03A444,1612841673,438A180AA7ABD7E6864B9D47406269C6,...,A351F2101260D57B20D6FA7C62977555,106,456,False,1549823766,False,<NA>,<NA>,<NA>,1.612841939e+09
4,101\t56898\t137\t64364\t10245\t10537\t61069\t1...,<NA>,3523FE3BD14D19AEF6DEF3D1E6CB8C6C,<NA>,<NA>,<NA>,Retweet,5B6973BEB05212E396F3F2DC6A31B71C,1612543661,C98077DF1DC01C3583C07BCD7E7ECC2C,...,E739A9F9465CACC9F0A2EC6CE666D32B,182,258,False,1593204248,True,<NA>,<NA>,<NA>,<NA>


In [5]:
df = df.drop('text_tokens', axis=1)
df, = dask.persist(df)
_ = wait(df)

In [6]:
%%time
df['id'] = 1
df['id'] = df['id'].cumsum()
df['id'] = df['id'].astype('int32')

df['reply'] = df['reply'].fillna(0)
df['retweet'] = df['retweet'].fillna(0)
df['retweet_comment'] = df['retweet_comment'].fillna(0)
df['like'] = df['like'].fillna(0)

df['reply'] = df['reply'].astype('int32')
df['retweet'] = df['retweet'].astype('int32')
df['retweet_comment'] = df['retweet_comment'].astype('int32')
df['like'] = df['like'].astype('int32')
df, = dask.persist(df)
_ = wait(df)

CPU times: user 765 ms, sys: 62.2 ms, total: 827 ms
Wall time: 4.49 s


In [7]:
%%time

df['timestamp'] = df['timestamp'].astype( np.int32 )
df['a_follower_count'] = df['a_follower_count'].astype( np.int32 )
df['a_following_count'] = df['a_following_count'].astype( np.int32 )
df['a_account_creation'] = df['a_account_creation'].astype( np.int32 )
df['b_follower_count'] = df['b_follower_count'].astype( np.int32 )
df['b_following_count'] = df['b_following_count'].astype( np.int32 )
df['b_account_creation'] = df['b_account_creation'].astype( np.int32 )

df, = dask.persist(df)
_ = wait(df)
df.head()

CPU times: user 350 ms, sys: 33.3 ms, total: 383 ms
Wall time: 2.74 s


,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id
0,<NA>,083AA40FC7D55123858207B610D3C06C,<NA>,<NA>,<NA>,TopLevel,E7F038DE3EAD397AEC9193686C911677,1612436125,AFC3FC2E4F720A36A9B7068E372AACA8,268,...,541,310,False,1351543406,True,0,0,0,1612443633,1
1,<NA>,C5E154739483D8733D68EC6FE8783C22,<NA>,<NA>,<NA>,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,1613276405,E27A4DB45665D5614B08EEE80A760EBA,1259,...,242,168,False,1348003745,True,0,0,0,1613278429,2
2,<NA>,759D744337DC3404CFC4E4DF43BA961F,<NA>,<NA>,<NA>,TopLevel,00304D7356D6C64481190D708D8F739C,1614077871,32DF697D49138DC2161A5413068734CD,1384552,...,320,59,False,1553742110,False,0,0,0,0,3
3,<NA>,C13243A3DADC6C2BCD4322100F3CC906,Video,<NA>,<NA>,Retweet,9FCF19233EAD65EA6E32C2E6DC03A444,1612841673,438A180AA7ABD7E6864B9D47406269C6,169,...,106,456,False,1549823766,False,0,0,0,1612841939,4
4,<NA>,3523FE3BD14D19AEF6DEF3D1E6CB8C6C,<NA>,<NA>,<NA>,Retweet,5B6973BEB05212E396F3F2DC6A31B71C,1612543661,C98077DF1DC01C3583C07BCD7E7ECC2C,2661,...,182,258,False,1593204248,True,0,0,0,0,5


In [8]:
%%time
tweet = df[['tweet_id']]
tweet = tweet.drop_duplicates(split_out=16)
tweet['tweet_encode'] = 1
tweet['tweet_encode'] = tweet['tweet_encode'].cumsum()
tweet, = dask.persist(tweet)
_ = wait(tweet)
tweet.head()

CPU times: user 267 ms, sys: 25.6 ms, total: 292 ms
Wall time: 2.2 s


,tweet_id,tweet_encode
101322,00009A6C1830C9A6DA073F5177442460,1
223116,00009C7E43293E86F8680795BC4B8E43,2
306636,0000C5CF4FAEBFCB687619C33D71D3A6,3
19816,0002896113EA31A3C04A4D7AC761E69B,4
422310,000305C4CAE2795063616F0106AD2A42,5


In [9]:
%%time
df = df.merge(tweet,on='tweet_id',how='left')
df = df.drop('tweet_id',axis=1)
df.columns = [i if i!='tweet_encode' else 'tweet_id' for i in df.columns]
df, = dask.persist(df)
wait(df)
del tweet
df.head()

CPU times: user 367 ms, sys: 31 ms, total: 398 ms
Wall time: 3.98 s


,hashtags,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,a_following_count,...,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,tweet_id
0,<NA>,<NA>,<NA>,<NA>,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1612724319,02F730EFB87C71908B73B91EAE733B8C,6195,4054,...,736,False,1566024576,True,0,0,0,0,157074,2454792
1,<NA>,<NA>,<NA>,<NA>,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1613720065,279C22E7900E24D974F30052C70357B7,2950,2544,...,2297,False,1439646864,True,0,0,0,0,163811,156241
2,CBC463F017C5B094034DA2E5812CEAD9,<NA>,<NA>,<NA>,TopLevel,E7F038DE3EAD397AEC9193686C911677,1613488313,9C1E8B67BB9428F0BD7CBC778B6A7F16,131019,399,...,273,False,1589542253,False,0,0,0,1613515046,161644,455138
3,<NA>,<NA>,714F7D3709E5465B4B530A7637A77FE4,9EFF000CDB18B710CDDB43EE1D8C300B,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,1613775677,9C56FE254DEFC8F2AE7A2366FFAA0CD6,27217,2059,...,794,False,1569632771,False,0,0,0,0,187602,365424
4,<NA>,<NA>,<NA>,<NA>,Retweet,5B6973BEB05212E396F3F2DC6A31B71C,1613111763,3A022DFD41D92FC4270784C51BD49308,10125,356,...,84,False,1586040001,False,0,0,0,1613117278,166556,2480338


In [10]:
%%time
user_a = df[['a_user_id']].drop_duplicates(split_out=16)
user_a, = dask.persist(user_a)
_ = wait(user_a)
user_b = df[['b_user_id']].drop_duplicates(split_out=16)
user_b, = dask.persist(user_b)
wait(user_b)

user_a.columns = ['user_id']
user_b.columns = ['user_id']
user_b['dummy'] = 1
user_a = user_a.merge(user_b,on='user_id',how='outer')
user_a = user_a.drop('dummy',axis=1)
user_a, = dask.persist(user_a)
wait(user_a)
del user_b

user_a['user_encode'] = 1
user_a['user_encode'] = user_a['user_encode'].cumsum()
user_a, = dask.persist(user_a)
_ = wait(user_a)

CPU times: user 721 ms, sys: 39.6 ms, total: 761 ms
Wall time: 3.78 s


In [11]:
%%time
df = df.merge(user_a,left_on='a_user_id',right_on='user_id',how='left')
df = df.drop(['a_user_id','user_id'],axis=1)
df.columns = [i if i!='user_encode' else 'a_user_id' for i in df.columns]
df, = dask.persist(df)
_ = wait(df)

CPU times: user 360 ms, sys: 29.5 ms, total: 390 ms
Wall time: 3.79 s


In [12]:
%%time
df = df.merge(user_a,left_on='b_user_id',right_on='user_id',how='left')
df = df.drop(['b_user_id','user_id'],axis=1)
df.columns = [i if i!='user_encode' else 'b_user_id' for i in df.columns]
df, = dask.persist(df)
wait(df)
del user_a
df.head()

CPU times: user 352 ms, sys: 17.2 ms, total: 369 ms
Wall time: 3.37 s


,hashtags,media,links,domains,tweet_type,language,timestamp,a_follower_count,a_following_count,a_is_verified,...,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,tweet_id,a_user_id,b_user_id
0,<NA>,<NA>,<NA>,<NA>,TopLevel,9A78FC330083E72BE0DD1EA92656F3B5,1613420925,7358,7092,False,...,1594071676,True,1613421255,0,0,0,5599690,1968467,189007,68770
1,<NA>,<NA>,<NA>,<NA>,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1613362941,6245,323,False,...,1563183247,False,0,0,0,1613372768,4953815,2090132,604310,3132939
2,<NA>,<NA>,<NA>,<NA>,Quote,488B32D24BD4BB44172EB981C1BCA6FA,1614196897,871571,1680,True,...,1234316138,False,0,0,0,0,8895451,1854707,1452322,495879
3,<NA>,<NA>,<NA>,<NA>,TopLevel,E7F038DE3EAD397AEC9193686C911677,1612841378,216,203,False,...,1591625960,True,0,0,0,0,4969487,1913268,1868135,2229891
4,5E9606D1A79963C3B1A9F010A67EBEF0,Photo,<NA>,<NA>,TopLevel,1F73BB863A39DB62B4A55B7E558DB1E8,1612690653,1851211,941,True,...,1594964294,False,0,0,0,1612852783,2446133,633642,197112,1303427


In [13]:
%%time
def count_token(ds,token):
    not_null = ds.isnull()==0
    return ((ds.str.count(token)+1)*not_null).fillna(0)
df['len_hashtags'] = df['hashtags'].map_partitions(lambda ds: count_token(ds,'\t'))
df['len_domains'] = df['domains'].map_partitions(lambda ds: count_token(ds,'\t'))
df['len_links'] = df['links'].map_partitions(lambda ds: count_token(ds,'\t'))

df, = dask.persist(df)
_ = wait(df)
df.head()

CPU times: user 162 ms, sys: 12.2 ms, total: 174 ms
Wall time: 769 ms


,hashtags,media,links,domains,tweet_type,language,timestamp,a_follower_count,a_following_count,a_is_verified,...,retweet,retweet_comment,like,id,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links
0,<NA>,<NA>,<NA>,<NA>,TopLevel,9A78FC330083E72BE0DD1EA92656F3B5,1613420925,7358,7092,False,...,0,0,0,5599690,1968467,189007,68770,0,0,0
1,<NA>,<NA>,<NA>,<NA>,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1613362941,6245,323,False,...,0,0,1613372768,4953815,2090132,604310,3132939,0,0,0
2,<NA>,<NA>,<NA>,<NA>,Quote,488B32D24BD4BB44172EB981C1BCA6FA,1614196897,871571,1680,True,...,0,0,0,8895451,1854707,1452322,495879,0,0,0
3,<NA>,<NA>,<NA>,<NA>,TopLevel,E7F038DE3EAD397AEC9193686C911677,1612841378,216,203,False,...,0,0,0,4969487,1913268,1868135,2229891,0,0,0
4,5E9606D1A79963C3B1A9F010A67EBEF0,Photo,<NA>,<NA>,TopLevel,1F73BB863A39DB62B4A55B7E558DB1E8,1612690653,1851211,941,True,...,0,0,1612852783,2446133,633642,197112,1303427,1,0,0


In [14]:
def factorize_small_cardinality(df,col):
    tmp_col = f'{col}_encode'
    tmp = df[col].unique().compute()
    tmp = tmp.to_frame().reset_index()
    tmp = tmp.rename(columns={'index':tmp_col})
    df = df.merge(tmp,on=col,how='left')
    df, = dask.persist(df)
    wait(df)
    del tmp
    df = df.drop(col,axis=1)
    df, = dask.persist(df)
    wait(df)
    df.columns = [i if i!=tmp_col else col for i in df.columns ]
    return df

In [15]:
%%time
for col in ['language','tweet_type']:
    df = factorize_small_cardinality(df,col)
    df[col] = df[col].astype('int8')

CPU times: user 350 ms, sys: 14.4 ms, total: 364 ms
Wall time: 2.41 s


In [16]:
df.head(20)

,hashtags,media,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,...,like,id,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links,language,tweet_type
0,<NA>,<NA>,<NA>,<NA>,1613507121,6035,4781,False,1550168946,1012,...,1613507600,5364633,1628230,564568,1781659,0,0,0,19,0
1,<NA>,<NA>,<NA>,<NA>,1613533453,1103,1641,False,1589394561,179,...,0,7507583,1349167,159477,958701,0,0,0,19,2
2,<NA>,<NA>,<NA>,<NA>,1613232247,19417,5,False,1397990573,1612,...,0,4726229,1405878,568255,930937,0,0,0,25,1
3,<NA>,Photo\tPhoto,<NA>,<NA>,1613313744,369576,1480,True,1270789011,2089,...,0,5668153,2561769,1048151,906436,0,0,0,60,1
4,<NA>,GIF,<NA>,<NA>,1612412487,15716,254,False,1459905071,21,...,0,87305,680251,1514503,513652,0,0,0,42,2
5,<NA>,<NA>,<NA>,<NA>,1613735611,229,222,False,1579087815,13,...,1613742852,3644618,2444167,1931488,17743,0,0,0,60,2
6,<NA>,Video,<NA>,<NA>,1613332656,389,334,False,1537485652,479,...,0,4887496,2724225,2734870,513856,0,0,0,45,1
7,<NA>,Photo,<NA>,<NA>,1613904836,156,264,False,1533705182,184,...,0,2656222,1323732,2368413,1054249,0,0,0,60,1
8,89748DC290574E1A20BE955C54F3FDAC,<NA>,8D90EC139CFD8688A84692C546A7EB63,594388EDA97514D6E194FBC318828177,1613373363,20759,4,False,1328507267,1,...,0,444760,1294199,997302,3105925,1,1,1,60,1
9,<NA>,<NA>,<NA>,<NA>,1612702345,123,131,False,1604926803,70,...,0,1153086,2408998,123063,2697954,0,0,0,60,2


In [17]:
def count_token(ds,token):
    not_null = ds.isnull()==0
    return ((ds.str.count(token))*not_null).fillna(0)

df['n_photo'] = df['media'].map_partitions(lambda ds: count_token(ds, 'Photo'))
df['n_gif'] = df['media'].map_partitions(lambda ds: count_token(ds, 'GIF'))
df['n_video'] = df['media'].map_partitions(lambda ds: count_token(ds, 'Video'))

df, = dask.persist(df)
_ = wait(df)
df.head()

,hashtags,media,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,...,a_user_id,b_user_id,len_hashtags,len_domains,len_links,language,tweet_type,n_photo,n_gif,n_video
0,<NA>,<NA>,<NA>,<NA>,1613507121,6035,4781,False,1550168946,1012,...,564568,1781659,0,0,0,19,0,0,0,0
1,<NA>,<NA>,<NA>,<NA>,1613533453,1103,1641,False,1589394561,179,...,159477,958701,0,0,0,19,2,0,0,0
2,<NA>,<NA>,<NA>,<NA>,1613232247,19417,5,False,1397990573,1612,...,568255,930937,0,0,0,25,1,0,0,0
3,<NA>,Photo\tPhoto,<NA>,<NA>,1613313744,369576,1480,True,1270789011,2089,...,1048151,906436,0,0,0,60,1,2,0,0
4,<NA>,GIF,<NA>,<NA>,1612412487,15716,254,False,1459905071,21,...,1514503,513652,0,0,0,42,2,0,1,0


In [20]:
df.head()

,hashtags,media,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,...,a_user_id,b_user_id,len_hashtags,len_domains,len_links,language,tweet_type,n_photo,n_gif,n_video
0,<NA>,<NA>,<NA>,<NA>,1613507121,6035,4781,False,1550168946,1012,...,564568,1781659,0,0,0,19,0,0,0,0
1,<NA>,<NA>,<NA>,<NA>,1613533453,1103,1641,False,1589394561,179,...,159477,958701,0,0,0,19,2,0,0,0
2,<NA>,<NA>,<NA>,<NA>,1613232247,19417,5,False,1397990573,1612,...,568255,930937,0,0,0,25,1,0,0,0
3,<NA>,Photo\tPhoto,<NA>,<NA>,1613313744,369576,1480,True,1270789011,2089,...,1048151,906436,0,0,0,60,1,2,0,0
4,<NA>,GIF,<NA>,<NA>,1612412487,15716,254,False,1459905071,21,...,1514503,513652,0,0,0,42,2,0,1,0
